In [16]:
# import packages

import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np

import math

# first, we need to import our essentia module.
import essentia

# as there are 2 operating modes in essentia which have the same algorithms,
# these latter are dispatched into 2 submodules:
import essentia.standard
import essentia.streaming

# # pylab contains the plot() function, as well as figure, etc... (same names as Matlab)
# from pylab import plot, show, figure, imshow
%matplotlib inline
import matplotlib.pyplot as plt

from essentia.standard import *


# import machine learning packages
import theano

import keras as keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, TimeDistributed, LSTM, Dropout, Activation
from keras.layers import Convolution2D, MaxPooling2D, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

from sklearn.metrics import confusion_matrix

import tensorflow as tf

import dill

import pydot as pydot

In [2]:
train = pd.read_pickle("jake_data/train_all.pkl")
test = pd.read_pickle("jake_data/test_all.pkl")

In [3]:
# pull out some data to make it better distributed
# train = train[0:21600]

In [4]:
# shuffle dataframe
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)

In [19]:
spectType = "melbands_log_file"

num_channels = 1

num_columns = 9999999999
for i in train[spectType]:
    if (len(i[0]) < num_columns):
        num_columns = len(i[0])

num_rows = len(train[spectType][0])

# need to figure out solution for different audio lenghts, jagged arrays?

X_train = []
Y_train = []
X_test = []
Y_test = []

X_test = np.zeros(shape=(len(test),1,num_rows,num_columns))
c = 0
for i in test[spectType]:
    l = i.shape[1]
    b = int(math.floor((l-num_columns)/2))
    tp = i[0:num_rows,b:(b + num_columns)]
    X_test[c,0,:,:] = tp
    c = c + 1
    
X_train = np.zeros(shape=(len(train),1,num_rows,num_columns))
c = 0
for i in train[spectType]:
    l = i.shape[1]
    b = int(math.floor((l-num_columns)/2))
    tp = i[0:num_rows,b:(b + num_columns)]
    X_train[c,0,:,:] = tp
    c = c + 1

Y_train = np.zeros(shape=(len(train),2)) 

c = 0
b = 0
s = 0
for i in train["output"]:
    if (i == "bonafide"):
        b = b + 1
        Y_train[c,:] = [np.float64(0.0),np.float64(1.0)]
    else:
        s = s + 1
        Y_train[c,:] = [np.float64(1.0),np.float64(0.0)]
    c = c + 1
print(b)
print(s)

Y_test = np.zeros(shape=(len(test),2)) 

c = 0
b = 0
s = 0
for i in test["output"]:
    if (i == "bonafide"):
        b = b + 1
        Y_test[c,:] = [np.float64(0.0),np.float64(1.0)]
    else:
        s = s + 1
        Y_test[c,:] = [np.float64(1.0),np.float64(0.0)]
    c = c + 1
       
print(b)
print(s)

5400
48600
5400
24300


In [20]:
X_train = X_train.reshape(X_train.shape[0], num_channels, num_rows, num_columns)
X_test = X_test.reshape(X_test.shape[0], num_channels, num_rows, num_columns)

num_labels = Y_train.shape[1]

model = Sequential()
model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(num_channels,num_rows,num_columns),data_format='channels_first'))

model.add(Convolution2D(32, 3, 3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(ELU(alpha=1.0)) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(32, 3, 3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(ELU(alpha=1.0)) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(32, 3, 3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(ELU(alpha=1.0)) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  if __name__ == '__main__':
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  app.launch_new_instance()
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`


In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 38, 123)       320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 36, 121)       9248      
_________________________________________________________________
batch_normalization_4 (Batch (None, 32, 36, 121)       128       
_________________________________________________________________
activation_4 (Activation)    (None, 32, 36, 121)       0         
_________________________________________________________________
elu_4 (ELU)                  (None, 32, 36, 121)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 18, 60)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 32, 18, 60)        0         
__________

In [32]:
import dill
from sklearn.utils import class_weight

# class_weights = {0: .99, 1: .01}

# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(y_train),
#                                                  y_train)

# class_weights = dict(enumerate(class_weights))
# print(class_weights)
# print(type(class_weights))

# compile to experement with better results, try not to get stuck at local min
# model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer='adam')

# normal compile
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

W0905 17:17:11.900672 4481955264 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

# we will want to add in seperated data to validate model
# we will also want to normalize data, see docs
# we may want to do multi channel learning

model.fit(X_train, Y_train,
               epochs=40,
               shuffle=True,
               batch_size=300,
#                class_weight=class_weights,
               verbose=1)

# model.fit(X_train, Y_train, 
#           batch_size=300, epochs=10, verbose=1)

Epoch 1/40
54000/54000 [==============================] - 509s 9ms/step - loss: 0.3821 - acc: 0.8794
Epoch 2/40
54000/54000 [==============================] - 497s 9ms/step - loss: 0.2901 - acc: 0.8996
Epoch 3/40
54000/54000 [==============================] - 492s 9ms/step - loss: 0.2666 - acc: 0.8999
Epoch 4/40
54000/54000 [==============================] - 488s 9ms/step - loss: 0.2473 - acc: 0.8999
Epoch 5/40
54000/54000 [==============================] - 488s 9ms/step - loss: 0.2366 - acc: 0.9000
Epoch 6/40
54000/54000 [==============================] - 490s 9ms/step - loss: 0.2273 - acc: 0.9000
Epoch 7/40
54000/54000 [==============================] - 489s 9ms/step - loss: 0.2185 - acc: 0.9000
Epoch 8/40
54000/54000 [==============================] - 489s 9ms/step - loss: 0.2137 - acc: 0.9000
Epoch 9/40
54000/54000 [==============================] - 489s 9ms/step - loss: 0.2070 - acc: 0.9000
Epoch 10/40
54000/54000 [==============================] - 489s 9ms/step - loss: 0.2012 - a

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print("testing accuracy " + str(score))

score = model.evaluate(X_train, Y_train, verbose=0)
print("training accuracy " + str(score))

In [ ]:
y_train = []
s = 0
b = 0
for i in Y_train:
    if (i[0] == 0.0):
        b = b + 1
        y_train.append(0)
    else:
        s = s + 1
        y_train.append(1)
 
y_train = np.array(y_train)
print("training set: ")
print("0 values " + str(b))
print("1 values " + str(s))

y_test = []
s = 0
b = 0

for i in Y_test:
    if (i[0] == 0.0):
        b = b + 1
        y_test.append(0)
    else:
        s = s + 1
        y_test.append(1)
 
y_test = np.array(y_test)
print("testing set: ")
print("0 values " + str(b))
print("1 values " + str(s))

In [ ]:
### 1 is bonafied, 0 is spoofed audio
train_predictions = model.predict(X_train)
confusion = confusion_matrix(y_train, np.argmax(train_predictions,axis=1))
print("training matrix: ")
print(confusion)

In [ ]:
### 1 is bonafied, 0 is spoofed audio
test_predictions = model.predict(X_test)
confusion = confusion_matrix(y_test, np.argmax(test_predictions,axis=1))
print("testing matrix: ")
print(confusion)